In [1]:
# Run DataExplore File to use datat in dataFrame
%run DataExplore.ipynb

Number of rows: 1599999
Number of columns: 6
Number of rows: 1599999
Number of columns: 6
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   ids     1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   flag    1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [2]:
# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords   
from nltk import PorterStemmer   
from nltk.tokenize import word_tokenize
# regular expression
import re      # matching the words in any content


# 2: Data Preprocessing

## 2.1: Selecting the text and target colum for our future analysis

In [3]:
# only two columns to need in futrure to sentiment analysis target and text
data = twitter_data[['target', 'text']]

## 2.2: Replacing the target's value  

######  Know that  here in target columns has two values 0 for negative and 4 for positive tweets, so here i'm replace the value 4 to 1

#### Assuming 1 to positive sentiment 4

In [4]:
# Replaceing the target value 4 to 1
data.replace({'target':{4:1}},inplace = True)
data['target'].value_counts()

C:\Users\KAMAL CHANDRA\AppData\Local\Temp\ipykernel_3984\2589728098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.replace({'target':{4:1}},inplace = True)


target
1    800000
0    799999
Name: count, dtype: int64

#### 0 ------>Negative Tweets

#### 1 ------>Positive Tweets

## 2.3 Printing unique value of target variables

In [5]:
# Check here all unique value store in target columns
data['target'].unique()

array([0, 1], dtype=int64)

## 2.4: Seprating positive and negative tweets

###### Dividing the data in two seprate parts 1's is positive and 2'nd is negative

In [7]:
# store only positive tweets here
data_pos = data[data['target'] == 1]
# -------------------------------------------
# Store only negative tweets in here
data_neg = data[data['target'] == 0]

In [8]:
# view top few positive tweets 
# data_pos.shape
data_pos.head()

,target,text
799999,1,I LOVE @Health4UandPets u guys r the best!!
800000,1,im meeting up with one of my besties tonight! ...
800001,1,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800002,1,Being sick can be really cheap when it hurts t...
800003,1,@LovesBrooklyn2 he has that effect on everyone


In [9]:
# view top few negative tweets
data_neg.shape
data_neg.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


# 2.5: Taking 10% data of twitter_data 

In [10]:
# half data are take for working 
data_pos = data_pos.iloc[:int(80000)]
data_neg = data_neg.iloc[:int(80000)]

## 2.6: Combilning positive and negative tweet

In [11]:
# Combined the both tweets postive and negative and store in dataset
dataset = pd.concat([data_neg,data_pos])
# view tp few tweets
dataset.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [12]:
# view last few tweets
dataset.tail()

,target,text
879994,1,"morn all, waitin 4 the sun 2 reappear, so cool..."
879995,1,Why am I still up at 3am? Good grief and good ...
879996,1,Morning world. Taking my parents out for a me...
879997,1,drive ins are fun...just saying.
879998,1,i like my twitter page background


In [13]:
dataset.shape

(160000, 2)

In [14]:
dataset['target'].value_counts() 

target
0    80000
1    80000
Name: count, dtype: int64

## 2: Stemming

In [15]:
# Download the stopwords dataset
# nltk.download('stopwords')

In [16]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## 2.1: Defined a function for Stemming of the text

In [13]:

# Instantiate the PorterStemmer
port_stem = PorterStemmer()


In [14]:
# Function 
def stemming(content):
    # Clean non-alphabetic characters and convert to lowercase
    cleaned_content = re.sub('[^a-zA-Z]', ' ', content)
    cleaned_content = cleaned_content.lower()

    # Tokenize the cleaned content
    tokenized_content = word_tokenize(cleaned_content)

    # Apply stemming and remove stopwords
    stemmed_content = [port_stem.stem(word)
                       for word in tokenized_content
                       if word.lower() not in stopwords.words('english')]

    # Join the stemmed words back into a single string
    stemmed_content = ' '.join(stemmed_content)
    
    return stemmed_content



In [15]:
# create a new column and apply steeming function
dataset['Stemmed_content'] = dataset['text'].apply(stemming)

# And print top few rows
dataset.head()

,target,text,Stemmed_content
0,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,@Kwesidei not the whole crew,kwesidei whole crew


In [19]:
dataset[['text','Stemmed_content']].head(10)

,text,Stemmed_content
0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,@Kwesidei not the whole crew,kwesidei whole crew
5,Need a hug,need hug
6,@LOLTrish hey long time no see! Yes.. Rains a...,loltrish hey long time see ye rain bit bit lol...
7,@Tatiana_K nope they didn't have it,tatiana k nope
8,@twittera que me muera ?,twittera que muera
9,spring break in plain city... it's snowing,spring break plain citi snow


In [20]:
# Assuming 'dataset' is your DataFrame
for index, row in dataset[['text', 'Stemmed_content']].head().iterrows():
    print(f"Original Text: {row['text']}")
    print(f"Stemmed Content: {row['Stemmed_content']}\n")


Original Text: is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
Stemmed Content: upset updat facebook text might cri result school today also blah

Original Text: @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
Stemmed Content: kenichan dive mani time ball manag save rest go bound

Original Text: my whole body feels itchy and like its on fire 
Stemmed Content: whole bodi feel itchi like fire

Original Text: @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
Stemmed Content: nationwideclass behav mad see

Original Text: @Kwesidei not the whole crew 
Stemmed Content: kwesidei whole crew



## 2.2: Applying lemmatizer

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\KAMAL
[nltk_data]     CHANDRA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [31]:

# Initialize the WordNetLemmatizer
lm = WordNetLemmatizer()

# Define function
def lemmatizing(content):
    # Tokenize the content
    tokens = nltk.word_tokenize(content)
    
    # Lemmatize each token
    lemmatized_tokens = [lm.lemmatize(token) for token in tokens]
    
    # Join the lemmatized tokens back into a single string
    lemmatized_content = ' '.join(lemmatized_tokens)
    lemmatized_content = ' '.join(lemmatized_tokens)
    
    return lemmatized_content

In [32]:
# # import nltk
# # from nltk.corpus import stopwords

# # nltk.download('wordnet')
# lm = nltk.WordNetLemmatizer()

# def lemmatizing(content):
#     text = [lm.lemmatize(word) for word in content 
#             if word not in dataset['Stemmed_content']]
# #     join the words
#     original_text = ' '.join(lemmatized_tokens)
# #     print(original_text)
#     return original_text



In [33]:
dataset['Lemmatize_content'] = dataset['Stemmed_content'].apply(lemmatizing)
dataset.head() 

,target,text,Stemmed_content,lemmatize_content,Lemmatize_content
0,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...,upsetupdatfacebooktextmightcriresultschooltoda...,upset updat facebook text might cri result sch...
1,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...,kenichandivemanitimeballmanagsaverestgobound,kenichan dive mani time ball manag save rest g...
2,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire,wholebodifeelitchilikefire,whole bodi feel itchi like fire
3,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see,nationwideclassbehavmadsee,nationwideclass behav mad see
4,0,@Kwesidei not the whole crew,kwesidei whole crew,kwesideiwholecrew,kwesidei whole crew


In [28]:
# Apply lemmatizer function
dataset['lemmatize_content'] = dataset['Stemmed_content'].apply(lemmatizing)


In [29]:
dataset.head()

,target,text,Stemmed_content,lemmatize_content
0,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...,upsetupdatfacebooktextmightcriresultschooltoda...
1,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...,kenichandivemanitimeballmanagsaverestgobound
2,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire,wholebodifeelitchilikefire
3,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see,nationwideclassbehavmadsee
4,0,@Kwesidei not the whole crew,kwesidei whole crew,kwesideiwholecrew


## Drop unnessacary columns

In [34]:
# columns_drop = ['text']
# dataset = dataset.drop(columns = 'text')
dataset = dataset.drop(columns = 'lemmatize_content')
dataset.head()

,target,text,Stemmed_content,Lemmatize_content
0,0,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...,upset updat facebook text might cri result sch...
1,0,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...,kenichan dive mani time ball manag save rest g...
2,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire,whole bodi feel itchi like fire
3,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see,nationwideclass behav mad see
4,0,@Kwesidei not the whole crew,kwesidei whole crew,kwesidei whole crew


### Make a csv file and store data

In [30]:
# Assuming your dataset is named 'dataset'
# dataset.to_csv('processed_data.csv', index=False)
# Store file in your system 
dataset.to_csv('processed_data_stemmed.csv', index = False)
# Save DataFrame to Excel
# dataset.to_excel('processed_data_table.xlsx', index=False)
